In [1]:
import pennylane as qml
import numpy as np
import torch
from torch.autograd import Variable
from torch.nn.functional import normalize
np.random.seed(42)

In [2]:
# number of qubits in the circuit
nr_qubits = 4
# number of layers in the circuit
nr_layers = 2
# number of parameters in the circuit
nr_params = 4

In [3]:
# function to create the normalized random input states
def create_input_state(qubits_states):

    if len(qubits_states) != nr_qubits:
        print('Error: expected {} qubits states, {} where provided.'.format(nr_qubits, len(qubits_states)))
        return
    
    normalized_states = [normalize(q, p=2, dim=0) for q in qubits_states]

    input_state = torch.kron(normalized_states[0], normalized_states[1])
    
    for i in range(2, nr_qubits):
        input_state = torch.kron(input_state, normalized_states[i])
    
    return input_state

In [4]:
# input and target states definition

zero = torch.tensor([1., 0.])
one  = torch.tensor([0., 1.])


input_states = {}

# random input states in the computational basis
for i in range(4):
    input_states[i] = create_input_state([torch.rand(2) + torch.rand(2)*1.j for _ in range(nr_qubits)])
'''
input_states[0] = create_input_state([one, zero, zero, zero])
input_states[1] = create_input_state([zero, one, zero, zero])
input_states[2] = create_input_state([zero, zero, one, zero])
input_states[3] = create_input_state([zero, zero, zero, zero])
'''

# desired output states, as represented on the bloch sphere
zero = [0., 0.,  1.]
one  = [0., 0., -1.]

target_states = {}

target_states[0] = torch.tensor([zero, zero, one, one])
target_states[1] = torch.tensor([zero, one, zero, one])
target_states[2] = torch.tensor([one, zero, one, zero])
target_states[3] = torch.tensor([one, one, zero, zero])

nr_states = len(target_states)

# normalization of each target state
for target_state in target_states.values():
    for i in range(len(target_state)):
        target_state[i] = normalize(target_state[i], p=2, dim=0)
        
# wrap input and target states in a useful structure
states = {}
for i in range(len(target_states)):
    states[i] = {'input': input_states[i],
                 'target': target_states[i]}


# array of Pauli matrices (will be useful later)
Paulis = Variable(torch.zeros([3, 2, 2], dtype=torch.complex128), requires_grad=False)
Paulis[0] = torch.tensor([[0, 1], [1, 0]])
Paulis[1] = torch.tensor([[0, -1j], [1j, 0]])
Paulis[2] = torch.tensor([[1, 0], [0, -1]])

In [5]:
# initialize parameters to random values
params = np.random.uniform(0, np.pi, (nr_qubits, nr_layers, nr_params))
params = Variable(torch.tensor(params), requires_grad=True)

In [6]:
# definition of one layer of the circuit
def rotation_layer(params, j):
    for i in range(nr_qubits):
        qml.RX(params[i, j, 0], wires=i)
        qml.RY(params[i, j, 1], wires=i)
        #qml.RZ(params[i, j, 2], wires=i)

        for k in range(i+1, nr_qubits):
            qml.CRX(params[i, j, 2], wires=[i, k])
            qml.CRY(params[i, j, 3], wires=[i, k])
            #qml.CRZ(params[i, j, 5], wires=[i, k])
            
def cnot_layer():
    for i in range(nr_qubits - 1):
        qml.CNOT(wires=[i, i+1])

In [7]:
def fixed_layer():
    for i in range(nr_qubits):
        qml.Hadamard(wires=i)
    
        for j in range(i+1, nr_qubits):
            qml.CNOT(wires=[i, j])

In [8]:
dev = qml.device("default.qubit", wires=nr_qubits)

In [9]:
@qml.qnode(dev, interface="torch")
def circuit(params, A, wire, input_state):

    qml.QubitStateVector(input_state, wires=range(nr_qubits))
    
    # apply layers in the circuit
    for j in range(nr_layers):
        rotation_layer(params, j)
        if j == 0:
            cnot_layer()
    
    
    # returns the expectation of the input matrix A on the given wire
    return qml.expval(qml.Hermitian(A, wires=wire))

In [10]:
# cost function for single computed and target couple states
def cost_fn_single_state(params, state):
    cost = 0
    for wire in range(nr_qubits):
        for k in range(3):
            cost += torch.abs(circuit(params, Paulis[k], wire, state['input']) - state['target'][wire][k])            

    return cost / nr_qubits

In [11]:
# cost function
# cost is the sum of the difference between each pair of computed and target states 
def cost_fn(params):
    cost = 0
    for state in states.values():
        cost += cost_fn_single_state(params, state)

    return cost / nr_states

In [12]:
total_steps = 0

In [ ]:
# Optimization
# set up the optimizer
opt = torch.optim.Adam([params], lr=0.1)

# number of steps in the optimization routine
steps = 50

# the final stage of optimization isn't always the best, so we keep track of
# the best parameters along the way
best_cost = cost_fn(params)
best_params = np.zeros((nr_qubits, nr_layers, nr_params))

# optimization begins
for n in range(steps):
    opt.zero_grad()
    loss = cost_fn(params)
    #print(loss)
    loss.backward()
    opt.step()
    
    # keeps track of best parameters
    if loss < best_cost:
        best_cost = loss
        best_params = params

    # Keep track of progress every 10 steps
    if n % 10 == 9 or n == steps - 1:
        print("Cost after {} steps is {:.4f}".format(total_steps + n + 1, loss))
    
params = best_params
total_steps += steps

Cost after 10 steps is 1.0570
Cost after 20 steps is 0.9948
Cost after 30 steps is 0.9701


In [ ]:
# calculate the Bloch vector of the output state
output_state = [torch.zeros((nr_qubits, 3)) for _ in range(nr_states)]
for key, state in states.items():
    for wire in range(nr_qubits):
        for l in range(3):
            output_state[key][wire][l] = circuit(best_params, Paulis[l], wire, state['input'])

    for i in range(len(state['target'])):
        output_state[key][i] = normalize(output_state[key][i], p=2, dim=0)

    # print results
    print('Results for input/target states number {}'.format(key+1))
    print("Target Bloch vector: \n", target_states[key])
    print("Output Bloch vector: \n", output_state[key])
    print("Cost = ", float(cost_fn_single_state(best_params, states[key])))
    print('\n')